**Simple Genetic algorithm for neuroevolution** 

This notebook aims to find a solution to flappy bird by using a simple genetic algorithm to conduct a large search to fit a neural network. The algorithm is based on genomes consisting of a bitstring, mapping the bits to a neural network. 

In [ ]:
# imports
%pip install flappy-bird-gym
import numpy as np
import random
import time
import flappy_bird_gym

The cell below contains functions for converting the chromosome into a neural network, as well as useful functions like forward pass in order to let the creatures make predictions. 

In [ ]:
# DO NOT MODIFY THIS CELL!

def bitstring_to_floats(bitstring, bits_per_value=10, min_val=-2, max_val=2):
    """Converts a bitstring into an array of float values while ensuring full extraction."""
    num_values = len(bitstring) // bits_per_value  # Ensure full number of weights
    if num_values == 0:
        raise ValueError("Bitstring too short for any weights!")

    floats = []
    for i in range(num_values):
        binary_segment = bitstring[i * bits_per_value:(i + 1) * bits_per_value]
        decimal_value = int(binary_segment, 2)  # Convert binary to decimal
        scaled_value = min_val + (max_val - min_val) * (decimal_value / (2**bits_per_value - 1))  # Normalize to [-2,2]
        floats.append(scaled_value)

    return np.array(floats)


def decode_architecture(bitstring, bits_per_weight=10, input_size=2, output_size=1):
    """Extracts NN structure (neurons) and weights from a bitstring"""
    hidden_neurons = int(bitstring[:4], 2) + 1  # Allow 1-16 hidden neurons
    weights = bitstring_to_floats(bitstring[4:], bits_per_value=bits_per_weight)

    # Compute expected number of weights
    required_params = (input_size * hidden_neurons) + hidden_neurons + (hidden_neurons * output_size) + output_size

    if len(weights) < required_params:
        raise ValueError(f"Decoded weights ({len(weights)}) are fewer than expected ({required_params}). "
                         f"Ensure bitstring is at least {required_params * bits_per_weight} bits long!")

    return hidden_neurons, weights


def construct_nn(bitstring, input_size=2, output_size=1):
    """Constructs a simple 1-layer NN from a bitstring with dynamic hidden neurons"""
    hidden_neurons, params = decode_architecture(bitstring, input_size=input_size, output_size=output_size)

    # Compute parameter indices
    w1_end = input_size * hidden_neurons
    b1_end = w1_end + hidden_neurons
    w2_end = b1_end + (hidden_neurons * output_size)

    # Extract weights and biases
    w1 = params[:w1_end].reshape(input_size, hidden_neurons)  # (input_size, hidden_neurons)
    b1 = params[w1_end:b1_end]  # (hidden_neurons,)
    w2 = params[b1_end:w2_end].reshape(hidden_neurons, output_size)  # (hidden_neurons, output_size)
    b2 = params[w2_end:w2_end + output_size]  # (output_size,)

    return w1, b1, w2, b2

def forward_pass(X, w1, b1, w2, b2):
    """Performs forward propagation through the neural network"""
    hidden = np.tanh(np.dot(X, w1) + b1)  # (batch, hidden)
    output = np.tanh(np.dot(hidden, w2) + b2)  # (batch, 1)

    return output

def required_bitstring_length(hidden_neurons=16, input_size=2, output_size=1, bits_per_weight=10):
    """Compute the required bitstring length for encoding all NN parameters."""
    required_params = (input_size * hidden_neurons) + hidden_neurons + (hidden_neurons * output_size) + output_size
    return required_params * bits_per_weight  # Convert to bitstring length



In [ ]:
# Save and load best genome of a run
def save_best_genome(genome, filename):
    """Saves the best genome to a file"""
    
    with open (filename, 'w') as f:
        f.write(genome)
        
def load_best_genome(filename):
    """Loads the best genome from a file"""
    
    with open (filename, 'r') as f:
        genome = f.read()
        
    return genome

## 1. **Initialize population**
Create a list of genes (bit strings) randomly for the entire population

In [ ]:
def initialize_population(pop_size, genome_length):
    """Initializes a population of genomes"""
    # TODO: return a random bitstring of length genome_length for each individual in the population

In [ ]:
# TEST
def test_initialize_population():
    np.random.seed(42)
    pop_size = 3
    genome_length = 5
    population = initialize_population(pop_size, genome_length)
    expected_population = ['01000', '10001', '00001']
    
    assert len(population) == pop_size, f"Expected population size {pop_size}, got {len(population)}"
    # Verify each genome has the correct length and contains only '0' and '1'
    for genome in population:
        assert len(genome) == genome_length, f"Expected genome length {genome_length}, got {len(genome)}"
        for char in genome:
            assert char in ['0', '1'], f"Unexpected character {char} in genome"

    assert population == expected_population, f"Expected {expected_population}, got {population}"
    print("All tests passed!")

test_initialize_population()

## **2. Fitness Function**
The fitness of a genome is calculated in a fitness function. This often the only domain centric part of a genetic algorithm, meaning that the neural network will be trained based on what kind of environment the fitness function uses.

In [ ]:


env = flappy_bird_gym.make("FlappyBird-v0")

def evaluate_fitness(chromosome):
    """Evaluates a chromosome based on its fitness."""
    hidden_neurons, params = decode_architecture(chromosome)
    w1, b1, w2, b2 = construct_nn(chromosome)

    # Run the game
    obs, _ = env.reset()
    total_reward = 0
    while True:
        
        try:
            X = np.array(obs).reshape(1, 2)   
        except:
            X = np.array([0.1,1.0]).reshape(1, 2) # the first observation is not in the correct shape

        # Forward pass through the NN
        output = forward_pass(X, w1, b1, w2, b2)

        action = 1 if output > 0.7 else 0

        # Take the action
        obs, reward, terminated, info = env.step(action)
        # TODO: update the total reward of the game.
        
        # Check if the game is over
        if terminated:
            break
        
    # TODO: return the total reward of the game + 1000 * the score
    #   Hint: clamp reward to be at least 0.000001




In [ ]:
# TEST 
def test_evaluate_fitness():
    np.random.seed(42)
    dummy_chromosome = "1011000110"*50
    fitness = evaluate_fitness(dummy_chromosome)
    assert isinstance(fitness, (int, float)), f"Expected fitness to be a number, got {type(fitness)}"
    assert fitness >= 0, f"Expected fitness to be non-negative, got {fitness}"
    print("Test passed!")

test_evaluate_fitness()

## **3. Selection**
Select what parents that will reproduce from the population. 
Different selection methods:
- tournament selection: choose random chromosomes from the population that will compete with each other. The winner may reproduce.
- roulette wheel selection: choose with a probability of (fitnesses/total fitness), increasing the chance for fit individuals to reproduce.


In [ ]:
def tournament_selection(population, fitnesses, tournament_size=10):
    """Select best genome from a random subset"""
    indices = # TODO: select tournament_size random indices from the population
    best_index = indices[np.argmax([fitnesses[i] for i in indices])]
    return population[best_index]


def roulette_wheel_selection(population, fitnesses):
    """Selects individuals using fitness proportionate selection."""
    total_fitness = # TODO: calculate the total fitness of the population
    probabilities = # TODO: calculate the probability of selecting each individual
    # (hint: read the markdown cell above for the formula)
    # TODO: return a random individual from the population based on the probabilities
    # hint: use random.choice: example: np.random.choice([2, 4, 6, 8], p=[0.1, 0.3, 0.6, 0.0])


## **4. Crossover**
The crossover function decides how different genomes should mate in order to produce offspring. 
The gene representation is in the form of a bitstring.

**Task** 
Define your own set of crossover functions by implementing different policies to combine the bitstirng of two parents. All genomes are the same length.

Diiferent kinds of crossover functions in this course:
- uniform crossover: define a random mask deciding what bits should be inherited from parent one and the rest should be from parent two. Reverse order for child 2



In [ ]:
def uniform_crossover(parent1, parent2):
    """Uniform crossover for binary genomes"""
    mask = np.random.randint(2, size=len(parent1))  # Random bit mask
    # TODO: use mask to combine parent1 and parent2 to create two children
    # hint: parent1 and parent2 are strings of 0s and 1s
    child1 = ""
    child2= ""
    return child1, child2


def single_point_crossover(parent1, parent2, crossover_rate=0.8):
    """Single-point crossover for binary genomes"""
    if np.random.rand() > crossover_rate:
        return parent1, parent2
    split_point = np.random.randint(1, len(parent1) - 1)  # Random split point
    # TODO: use split_point to combine parent1 and parent2 to create two children
    # hint: parent1 and parent2 are strings of 0s and 1s
    # hint: child1 should be parent1 up to the split_point and parent2 from the split_point onwards
    child1 = ""
    child2= ""
    return child1, child2


def crossover(parent1, parent2, crossover_rate=0.8):
    random_number = np.random.rand()
    if random_number < 0.5:
        return single_point_crossover(parent1, parent2, crossover_rate)
    else:
        return uniform_crossover(parent1, parent2)



## **5. Mutation**
Mutate genes in order to introduce new features to the phenotype. Types of mutations in this course:
- bit flip mutation: for each gene (bit) flip it if random number is below mutation rate.
- swap mutation: swap two genes at random positions


In [ ]:
def bit_flip_mutation(chromosome, mutation_rate=0.01):
    """Flips random bits with given probability"""
    # TODO: flip each bit in the bitstring with probability mutation_rate
    return chromosome



def swap_mutation(chromosome):
    """Performs swap mutation on a given chromosome by swapping two random genes."""
    if len(chromosome) < 2:
        return chromosome  # No mutation possible
    
    idx1, idx2 = random.sample(range(len(chromosome)), 2)
    # TODO: swap the genes at idx1 and idx2 in the chromosome
    
    return chromosome


def adaptive_mutate(chromosome, generation, max_generations, mutation_rate=0.01, diversity_factor=1):
    """Adapts mutation rate based on population diversity."""
    adjusted_mutation_rate = mutation_rate * (1 - (generation / max_generations)) * diversity_factor

    chromosome_list = list(chromosome)  # Convert to mutable list
    for i in range(len(chromosome_list)):
        if np.random.rand() < adjusted_mutation_rate:
            chromosome_list[i] = '1' if chromosome_list[i] == '0' else '0'  # Flip bit

    return "".join(chromosome_list)  # Convert back to string


In [ ]:
def genetic_algorithm(pop_size, genome_length, generations):
    """Runs a genetic algorithm to evolve a neural network"""
    # Initialize population (random bitstrings)
    population = initialize_population(pop_size, genome_length)
    max_generation = population

    for gen in range(generations):
        fitnesses = np.array([evaluate_fitness(ind) for ind in population]) 
        
        new_population = []
        # TODO: Elitism saves the top 20 % of the population. Play around with this number and see if you can get a better model
        # keep the 20% best genomes
        new_population.extend([population[i] for i in np.argsort(fitnesses)[-int(pop_size * 0.2):]])
        while len(new_population) < pop_size:
            # Select parents
            #parent1, parent2 = tournament_selection(population, fitnesses), tournament_selection(population, fitnesses)
            parent1 = roulette_wheel_selection(population, fitnesses)
            parent2 = roulette_wheel_selection(population, fitnesses)
            # TODO: selection
            parent1 = # Choose your selection method or choose randomly among the methods you have implemented
            parent2 = # Choose your selection method or choose randomly among the methods you have implemented
            
            # TODO: Crossover
            child1, child2 = # Choose your crossover method or choose randomly among the methods you have implemented
            
            # TODO: Mutation
            child1 = # Choose your mutation method or choose randomly among the methods you have implemented
            child2 = # Choose your mutation method or choose randomly among the methods you have implemented
            
            # if you want to use adaptive mutate (a bit slower than the other methods):
            #diversity_factor = 1 + (np.mean(fitness_sharing(population, fitnesses)) * 0.5)  # Scale mutation if diversity is low
            #child1 = adaptive_mutate(child1, gen, generations, diversity_factor=diversity_factor)
            #child2 = adaptive_mutate(child2, gen, generations, diversity_factor=diversity_factor)
            
            # if you want to use deterministic crowding (a bit slower than the other methods, but very good for diversity when a large population is used):
            #child1, child2 = deterministic_crowding(parent1, parent2, child1, child2, evaluate_fitness)
            
            new_population.extend([child1, child2])

        # Replace population (keep best elite)
        best_idx = np.argmax(fitnesses)
        best_genome = population[best_idx]
        population = new_population[:pop_size]
        population[0] = best_genome  # Elitism
        if population[np.argmax(fitnesses)] == best_genome:
            max_generation = population.copy()
        
        print(f"Generation {gen + 1}: Best Fitness = {max(fitnesses):.6f} avg fitness = {np.mean(fitnesses):.6f}")
       
    
    best_genome = max_generation[np.argmax(fitnesses)]
    print(f"Best genome: {best_genome}")
    return best_genome  # Return best genome


In [ ]:


input_size = 2
output_size = 1
hidden_neurons = 16  # Max number of hidden neurons
bits_per_weight = 10

required_length = required_bitstring_length(hidden_neurons, input_size, output_size, bits_per_weight) +10

print(f"Required bitstring length: {required_length} bits")  # Debugging

best_genome = genetic_algorithm(
    pop_size= 1,  # TODO: choose a population size (e.g., 100),but play around with this number and see if you can get a better model
    genome_length=required_length,  # 16 parameters × 8 bits
    generations=1 # TODO: choose a number of generations (e.g., 100)
    
)
# !!! important, delete best_genome.txt before running the code again, otherwise genomes may be combined and the code will not work as expected
save_best_genome(best_genome, "best_genome.txt") # Save best genome to file






In [ ]:

import random
env = flappy_bird_gym.make("FlappyBird-v0")

num_inputs = 2 # The envirement has 12 observations for each frame
num_outputs = 1 # The envirement has 1 action space (flap or do nothing)
    
#best_genome = load_best_genome("best_genome_score_275.txt")
w1, b1, w2, b2 = construct_nn(best_genome, input_size=num_inputs, output_size=num_outputs)
total_reward = 0
best_score = 0
runs = 100
total_score = 0
for i in range(runs):
    obs, _ = env.reset()
    while True:
        try:
            X = np.array(obs).reshape(1, 2)
        except:
            X = np.array([0.1,1.0]).reshape(1, 2)
        # Forward pass through the NN
        output = forward_pass(X, w1, b1, w2, b2)
        action = 1 if output > 0.7 else 0
        obs, reward, terminated, info = env.step(action)
        
        if terminated:
            break
    print(info)
   
    total_score += info['score']
    
print("avg score: ", total_score/runs)   

env.close()





In [ ]:

import random
env = flappy_bird_gym.make("FlappyBird-v0")


# ===============================================================
# Neural network parameters:
# ===============================================================
num_inputs = 2 # The envirement has 12 observations for each frame
num_outputs = 1 # The envirement has 1 action space (flap or do nothing)
    
best_genome = load_best_genome("best_genome_score_275.txt")
w1, b1, w2, b2 = construct_nn(best_genome, input_size=num_inputs, output_size=num_outputs)
obs, _ = env.reset()
total_reward = 0
while True:
    
    
    try:
        X = np.array(obs).reshape(1, 2)
    except:
        X = np.array([0.1,1.0]).reshape(1, 2)

    # Forward pass through the NN
    output = forward_pass(X, w1, b1, w2, b2)
    action = 1 if output > 0.7 else 0
    obs, reward, terminated, info = env.step(action)
    total_reward += reward
    
    env.render()
    time.sleep(1 / 30)
    
    if terminated:
        break
print(info)
    
  

env.close()





In [ ]:
# Uncomment the code below to visualize the neural network

# import pygame
# import numpy as np

# def draw_neural_network(chromosome, screen_size=(800, 600)):
#     """
#     Draws a neural network based on the given chromosome using Pygame.

#     Parameters:
#     - chromosome: Bitstring representing the neural network structure.
#     - screen_size: Tuple (width, height) for pygame window.
#     """
#     # Decode the architecture and construct the neural network
#     hidden_neurons, params = decode_architecture(chromosome)
#     w1, b1, w2, b2 = construct_nn(chromosome)

#     # Ensure hidden_neurons is a list
#     input_neurons = 2  # From the Flappy Bird environment
#     output_neurons = 1  # Single output neuron
#     layers = [input_neurons] + ([hidden_neurons] if isinstance(hidden_neurons, int) else hidden_neurons) + [output_neurons]

#     pygame.init()
#     screen = pygame.display.set_mode(screen_size)
#     pygame.display.set_caption("Neural Network Visualization")

#     # Colors
#     WHITE = (255, 255, 255)
#     BLACK = (0, 0, 0)
#     BLUE = (50, 150, 255)
#     GREEN = (100, 255, 100)
#     RED = (255, 50, 50)

#     screen.fill(WHITE)

#     # Define spacing
#     width, height = screen_size
#     layer_spacing = width // (len(layers) + 1)  # Space between layers
#     node_radius = 20  # Neuron size
#     y_padding = 80  # Padding from top/bottom

#     # Compute node positions
#     node_positions = []  # List of lists for neuron positions

#     for i, neurons in enumerate(layers):
#         x = (i + 1) * layer_spacing
#         y_spacing = (height - 2 * y_padding) // (neurons - 1) if neurons > 1 else 200

#         layer_nodes = []
#         for j in range(neurons):
#             y = y_padding + j * y_spacing
#             layer_nodes.append((x, y))

#         node_positions.append(layer_nodes)

#     # Draw connections (edges) with weight intensity
#     for i in range(len(layers) - 1):
#         weight_matrix = w1 if i == 0 else w2  # Use w1 for input-hidden, w2 for hidden-output
#         layer1_nodes = node_positions[i]
#         layer2_nodes = node_positions[i + 1]

#         for j, node1 in enumerate(layer1_nodes):  # Nodes in layer i
#             for k, node2 in enumerate(layer2_nodes):  # Nodes in layer i+1
#                 try:
#                     weight = weight_matrix[k, j]  # Extract weight
#                     color = GREEN if weight > 0 else RED  # Green for positive, red for negative
#                     thickness = int(abs(weight) * 5) + 1  # Scale thickness
#                 except:
#                     pass
                    
#                 pygame.draw.line(screen, color, node1, node2, thickness)

#     # Draw nodes (neurons)
#     for layer in node_positions:
#         for node in layer:
            
#             pygame.draw.circle(screen, BLUE, node, node_radius)
#             pygame.draw.circle(screen, BLACK, node, node_radius, 2)  # Outline

#     # Display network
#     pygame.display.flip()

#     # Wait until user closes the window
#     running = True
#     while running:
#         for event in pygame.event.get():
#             if event.type == pygame.QUIT:
#                 running = False

#     pygame.quit()



# draw_neural_network(load_best_genome("best_genome.txt"))